#### 한국어 언어모델 다분야 사고력 벤치마크 (MT-Bench)
- https://github.com/instructkr/LogicKor/tree/main 참고중

In [1]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
# - OPENAI_API_KEY = ""
# - LANGCHAIN_TRACING_V2 = "true"
# - LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
# - LANGCHAIN_API_KEY = ""
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study" # Langsmith project 명

In [2]:
# 비교대상 모델ID 상수 정의
IS_HOME = True

if IS_HOME:
    AYA = "aya:8b-23-q8_0"
    LLAMA3 = "llama3:8b-instruct-q8_0"
    GEMMA = "gemma:7b-instruct-q8_0"
    PHI3 = "phi3:14b-medium-4k-instruct-q4_1"
else:
    AYA = "aya"
    LLAMA3 = "llama3:instruct"
    GEMMA = "gemma"
    PHI3 = "phi3:instruct"

GPT = "gpt-4o"

In [3]:
from langchain_community.llms import Ollama

llm_aya = Ollama(model=AYA, temperature=0)
llm_llama3 = Ollama(model=LLAMA3, temperature=0)
llm_gemma = Ollama(model=GEMMA, temperature=0)
llm_phi3 = Ollama(model=PHI3, temperature=0)


In [4]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(model=GPT, temperature=0)

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    System : 너는 다음 Instruction을 잘 수행하는 assistant 이다.
    Instruction : {instruction}
    """
)

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [7]:
chain_aya = prompt | llm_aya | output_parser
chain_llama3 = prompt | llm_llama3 | output_parser
chain_gemma = prompt | llm_gemma | output_parser
chain_phi3 = prompt | llm_phi3 | output_parser
chain_gpt = prompt | llm_gpt | output_parser

param = {
    AYA : chain_aya ,
    LLAMA3 : chain_llama3 ,
    GEMMA : chain_gemma ,
    PHI3 : chain_phi3 ,
    GPT : chain_gpt ,
}

from langchain_core.runnables import RunnableParallel

chain_llms = RunnableParallel(**param)

#### (TEST) 평가대상 LLM 문제 출제 및 답안 작성

In [8]:
# chain_llms 실행 Test
responses = chain_llms.invoke({"instruction":"식혜라는 음료수에 대해 알려주세요."})

from pprint import pprint
pprint(responses)

{'aya:8b-23-q8_0': '식혜는 한국의 전통 음료로, 쌀을 사용하여 만든 달콤하고 고소한 맛이 특징입니다. 일반적으로 쌀을 물에 '
                   '담가 불려서 갈아 만든 후, 설탕과 소금을 넣어 단맛과 짠맛을 내고, 때로는 우유나 크림을 넣어 부드러운 '
                   '맛을 더하기도 합니다. 식혜는 한국에서는 여름철에 시원한 음료로 인기가 많으며, 일반적으로 병이나 캔에 '
                   '담겨 판매됩니다. 또한, 식혜는 영양가가 높아서 단백질과 칼슘이 풍부하고, 쌀가루를 사용하여 만들어지기 '
                   '때문에 탄수화물도 함유하고 있습니다.',
 'gemma:7b-instruct-q8_0': '## 식혜에 대한 설명\n'
                           '\n'
                           '식혜는 한국의 전통 음료수입니다. 주요 성분은 물, 쌀, 콩, 엿, 설탕입니다. 맛은 '
                           '달콤하고, 향은 약간 강렬합니다.\n'
                           '\n'
                           '식혜는 다음과 같은 주요 성분으로 구성됩니다.\n'
                           '\n'
                           '* **쌀:** 식혜의 주요 성분인 쌀은 영양소와 비타민 B1을 함유하고 있습니다.\n'
                           '* **콩:** 콩은 단백질, 비타민 B2와 칼슘을 함유하고 있습니다.\n'
                           '* **엿:** 엿은 식혜의 주요 향기와 맛을 주는 중요한 성분입니다.\n'
                           '* **설탕:** 식혜의 주요 맛을 주는 설탕입니다.\n'
                           '\n

In [15]:
#instruction = "1990년대 한국 대중가요에 대해 알려주세요."
#instruction = "필리핀의 대표적인 대중교통 알려줘"
instruction = """
강수는 초등학교 5학년으로, 그의 하루는 매우 바쁘고 다채롭다. 다음은 강수의 일상을 시간 순서대로 이야기한 것이다.
강수의 하루를 100자 내외로 요약하라.

---
아침
7:00 AM
알람 소리에 잠을 깬 강수는 이불을 박차고 일어난다. 세수를 하고 이를 닦은 후, 엄마가 차려 주신 아침 식사를 먹는다. 오늘 아침은 계란말이와 김치, 그리고 따뜻한 국이다.

7:30 AM
아침을 먹고 나서 교복을 입고 가방을 챙긴다. 가방에는 교과서, 공책, 필통, 그리고 어제 한 숙제들이 들어 있다. 준비가 다 되면 엄마에게 "다녀오겠습니다!"라고 인사한 후 집을 나선다.

학교
8:00 AM
학교에 도착한 강수는 친구들과 인사를 나누고 교실로 들어간다. 8시 10분이 되면 아침 조회가 시작된다. 선생님께서 오늘의 공지 사항을 말씀하시고, 학생들은 이를 조용히 듣는다.

9:00 AM - 12:00 PM
오전 수업이 시작된다. 오늘은 국어, 수학, 과학, 그리고 체육 시간이 있다. 강수는 특히 과학 시간이 재미있다. 오늘은 실험을 하는 날이라 더욱 기대된다. 수업 시간 동안 친구들과 함께 다양한 활동을 하며 즐겁게 배우고, 질문도 많이 한다.

점심
12:00 PM - 1:00 PM
점심시간이 되면 친구들과 함께 급식실로 향한다. 오늘의 메뉴는 된장찌개와 불고기다. 강수는 배가 고파서 맛있게 밥을 먹는다. 점심을 다 먹고 나면 운동장에서 친구들과 축구를 한다. 점심시간은 강수에게 가장 신나는 시간 중 하나이다.

오후
1:00 PM - 3:00 PM
오후 수업이 시작된다. 사회, 미술, 그리고 영어 시간이 있다. 강수는 영어 시간이 조금 어렵지만, 선생님이 재미있게 가르쳐 주셔서 흥미를 가지고 열심히 듣는다. 미술 시간에는 자신의 창의력을 발휘하여 그림을 그리며 즐거운 시간을 보낸다.

3:00 PM
수업이 끝나고 방과 후 활동 시간이 된다. 강수는 로봇 공학 동아리에 가입해 있어서 로봇을 만드는 활동을 한다. 오늘은 로봇 팔을 조립하는 법을 배운다.

저녁
4:30 PM
방과 후 활동이 끝나고 집으로 돌아온다. 집에 도착하면 엄마가 간식을 준비해 주신다. 과일과 우유를 먹으며 잠시 쉬는 시간을 가진다.

5:00 PM - 6:30 PM
간식을 먹고 나면 숙제를 한다. 오늘 배운 내용을 복습하며 수학 문제를 풀고, 영어 단어를 외운다. 숙제가 끝나면 읽고 싶은 책을 읽으며 시간을 보낸다.

6:30 PM
저녁 시간이 되어 가족들과 함께 저녁을 먹는다. 저녁을 먹으며 오늘 하루 있었던 일들을 이야기한다. 가족들과의 대화는 강수에게 매우 소중한 시간이다.

밤
8:00 PM
저녁을 먹고 나면 잠시 TV를 보거나 게임을 한다. 그런 후에는 샤워를 하고, 내일의 준비물을 챙긴다.

9:00 PM
잠자기 전, 부모님과 함께 하루를 마무리하는 이야기를 나눈다. 그리고 책을 읽거나 음악을 들으며 마음을 진정시킨다.

9:30 PM
잠자리에 든다. 내일을 위해 충분히 쉬기 위해 스스로 알람을 맞추고, 곧 깊은 잠에 빠진다.

강수의 하루는 이렇게 끝이 난다. 매일매일 새로운 것을 배우고, 친구들과 즐거운 시간을 보내며 바쁘지만 행복한 일상을 보내고 있다.
"""

In [16]:
system_prompt = """
너는 질문에 대한 한국어 언어 모델들의 답변을 매우 꼼꼼히 평가할 거야. 최대로 공정한 평가를 하기 위해서는 아래 몇 가지 규칙을 준수해야 해.

# 기본 규칙
1. 답변의 정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness)에 집중하여 평가할 것
2. 질문의 요구에서 놓친 것이 있는지 상세히 분석할 것
3. 답변의 길이가 평가에 영향을 미치게 하지 말 것
4. 만약 Ground Truth가 주어진다면 평가 시 해당 사실을 참고할 것

Instruction과 Responses 안의 각각의 llm별 응답을 
정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

한국어로 답변해줘.
"""


#### CASE # 1. StrOutputParser

In [19]:
eval_prompt = PromptTemplate(
    template="""

    System : {system_prompt}
    Instruction : {instruction}
    Resonses : {responses}
    """,
    input_variables=["instruction", "responses"],
    partial_variables={"system_prompt" : system_prompt},
)

In [20]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt
    | llm_gpt
    | output_parser
)

- 실행(invoke)

In [21]:
response = chain_combinded.invoke({"instruction":instruction})
print(response)

### 평가 기준

1. **정확성(Accuracy)**: 답변이 주어진 정보와 얼마나 일치하는지 평가합니다.
2. **관련성(Relevance)**: 답변이 질문에 얼마나 적절하게 대응하는지 평가합니다.
3. **유창성(Fluency)**: 답변이 문법적으로 얼마나 자연스럽고 읽기 쉬운지 평가합니다.
4. **완전성(Completeness)**: 답변이 질문의 요구를 얼마나 완벽하게 충족하는지 평가합니다.

### 평가

#### aya:8b-23-q8_0
- **정확성**: 5점
  - 강수의 하루 일과를 정확하게 요약하고 있습니다.
- **관련성**: 5점
  - 질문에 대한 답변으로 매우 적절합니다.
- **유창성**: 5점
  - 문장이 자연스럽고 읽기 쉽습니다.
- **완전성**: 5점
  - 강수의 하루를 아침부터 밤까지 모두 포함하여 잘 요약했습니다.

**총점: 20점**

#### llama3:8b-instruct-q8_0
- **정확성**: 4점
  - 전반적으로 정확하지만, 일부 세부 사항이 생략되었습니다.
- **관련성**: 4점
  - 질문에 대한 답변으로 적절하지만, 영어로 작성되어 있습니다.
- **유창성**: 4점
  - 영어로 작성되어 있어 한국어 평가 기준에 맞지 않습니다.
- **완전성**: 4점
  - 주요 활동을 포함하고 있지만, 일부 세부 사항이 빠져 있습니다.

**총점: 16점**

#### gemma:7b-instruct-q8_0
- **정확성**: 4점
  - 전반적으로 정확하지만, 일부 세부 사항이 생략되었습니다.
- **관련성**: 4점
  - 질문에 대한 답변으로 적절하지만, 일부 내용이 부족합니다.
- **유창성**: 4점
  - 문장이 자연스럽지만, 일부 오타가 있습니다.
- **완전성**: 4점
  - 주요 활동을 포함하고 있지만, 일부 세부 사항이 빠져 있습니다.

**총점: 16점**

#### phi3:14b-medium-4k-instruct-q4_1
- **정확성**: 1점
  - 답변이 질문과

#### CASE # 2. PydanticOutputParser

In [28]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class EvaluationByModel(BaseModel):
    model_id: str = Field(description="LLM 모델 이름 또는 LLM 모델 ID")
    accuracy_eval: str = Field(description="정확성(Accuracy) 평가")
    accuracy_score: int = Field(description="정확성(Accuracy) 평가 점수")
    relevance_eval: str = Field(description="관련성(Relevance) 평가")
    relevance_score: int = Field(description="관련성(Relevance) 평가 점수")
    fluency_eval: str = Field(description="유창성(Fluency) 평가")
    fluency_score: int = Field(description="유창성(Fluency) 평가 점수")
    completeness_eval: str = Field(description="완전성(Completeness) 평가")
    completeness_score: int = Field(description="완전성(Completeness) 평가 점수")

class EvaluationResponse(BaseModel):
    instruction: str = Field(description="Instruction 내용 전체")
    accuracy: str = Field(description="정확성(Accuracy) 평가 기준")
    relevance: str = Field(description="관련성(Relevance) 평가 기준")
    fluency: str = Field(description="유창성(Fluency) 평가 기준")
    completeness: str = Field(description="완전성(Completeness) 평가 기준")
    evaluation_by_model: List[EvaluationByModel] = Field(description="LLM 모델별 세부 평가 내용")
    overall: str = Field(description="종합평가")

In [29]:
# Schema 확인
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
f = convert_pydantic_to_openai_function(EvaluationResponse)
f

c:\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'EvaluationResponse',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'instruction': {'description': 'Instruction 내용 전체',
    'type': 'string'},
   'accuracy': {'description': '정확성(Accuracy) 평가 기준', 'type': 'string'},
   'relevance': {'description': '관련성(Relevance) 평가 기준', 'type': 'string'},
   'fluency': {'description': '유창성(Fluency) 평가 기준', 'type': 'string'},
   'completeness': {'description': '완전성(Completeness) 평가 기준',
    'type': 'string'},
   'evaluation_by_model': {'description': 'LLM 모델별 세부 평가 내용',
    'type': 'array',
    'items': {'type': 'object',
     'properties': {'model_id': {'description': 'LLM 모델 이름 또는 LLM 모델 ID',
       'type': 'string'},
      'accuracy_eval': {'description': '정확성(Accuracy) 평가', 'type': 'string'},
      'accuracy_score': {'description': '정확성(Accuracy) 평가 점수',
       'type': 'integer'},
      'relevance_eval': {'description': '관련성(Relevance) 평가', 'type': 'string'},
      'relevance_score': {'description': '관련성(Relevance)

In [30]:
from langchain.output_parsers import PydanticOutputParser

p_parser = PydanticOutputParser(pydantic_object=EvaluationResponse)

In [31]:
eval_prompt = PromptTemplate(
    template="""

    System : {system_prompt}
    Instruction : {instruction}
    Responses : {responses}

    Format : 
    {format}
    """,
    input_variables=["instruction", "responses"],
    partial_variables={"system_prompt": system_prompt, "format" : p_parser.get_format_instructions()},
)

In [32]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt 
    | llm_gpt
    | p_parser
)

- 실행(invoke)

In [33]:
response = chain_combinded.invoke({"instruction":instruction})
response.instruction = instruction # instruction 원문을 편집하는 경우가 있어 원래 instruction 값으로 update
print(response)


instruction='강수는 초등학교 5학년으로, 그의 하루는 매우 바쁘고 다채롭다. 다음은 강수의 일상을 시간 순서대로 이야기한 것이다.\n강수의 하루를 100자 내외로 요약하라.' accuracy='정확성(Accuracy) 평가 기준' relevance='관련성(Relevance) 평가 기준' fluency='유창성(Fluency) 평가 기준' completeness='완전성(Completeness) 평가 기준' evaluation_by_model=[EvaluationByModel(model_id='aya:8b-23-q8_0', accuracy_eval='강수의 하루를 잘 요약했으나, 100자 내외의 요구를 초과함.', accuracy_score=4, relevance_eval='질문과 관련된 내용을 잘 다룸.', relevance_score=5, fluency_eval='문장이 자연스럽고 유창함.', fluency_score=5, completeness_eval='강수의 하루를 전반적으로 잘 설명했으나, 요약이 길어짐.', completeness_score=4), EvaluationByModel(model_id='llama3:8b-instruct-q8_0', accuracy_eval='강수의 하루를 잘 요약했으나, 영어로 작성됨.', accuracy_score=2, relevance_eval='질문과 관련된 내용을 다루었으나, 언어가 맞지 않음.', relevance_score=2, fluency_eval='영어로 작성되어 유창성 평가가 어려움.', fluency_score=1, completeness_eval='강수의 하루를 전반적으로 설명했으나, 언어가 맞지 않음.', completeness_score=2), EvaluationByModel(model_id='gemma:7b-instruct-q8_0', accuracy_eval='강수의 하루를 요약했으나, 일부 문장이 부자연스러움.', accuracy_score=3, relevance_eval='질문

#### 질문과 평가결과를 DB에 저장하기

- CASE #1. 명시적으로 Table 만들기

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

class TEvaluationResponse(Base):
    __tablename__ = 'evaluation_responses'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    instruction = Column(Text, nullable=False)
    accuracy = Column(Text, nullable=False)
    relevance = Column(Text, nullable=False)
    fluency = Column(Text, nullable=False)
    completeness = Column(Text, nullable=False)
    overall = Column(Text, nullable=False)
    
    evaluations = relationship("TEvaluationByModel", back_populates="response")

class TEvaluationByModel(Base):
    __tablename__ = 'evaluation_by_model'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    model_id = Column(String, nullable=False)
    accuracy_eval = Column(Text, nullable=False)
    accuracy_score = Column(Integer, nullable=False)
    relevance_eval = Column(Text, nullable=False)
    relevance_score = Column(Integer, nullable=False)
    fluency_eval = Column(Text, nullable=False)
    fluency_score = Column(Integer, nullable=False)
    completeness_eval = Column(Text, nullable=False)
    completeness_score = Column(Integer, nullable=False)
    
    response_id = Column(Integer, ForeignKey('evaluation_responses.id'))
    response = relationship("TEvaluationResponse", back_populates="evaluations")

# Create an engine and a session
engine = create_engine('sqlite:///evaluations.db')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)


- (Error 남)CASE #2. Pydantic Schema를 이용하여 동적으로 Table 만들기 (ChatGPT가 만들어 줌)

In [15]:
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from typing import Dict, Type

Base = declarative_base()

# Mapping Pydantic types to SQLAlchemy types
type_mapping = {
    str: String,
    int: Integer,
    float: Text,
}

def create_sqlalchemy_model(pydantic_model: Type[BaseModel], base: Type[Base], table_name: str, exclude_fields: List[str] = None, additional_columns: Dict[str, Type[Column]] = None):
    if exclude_fields is None:
        exclude_fields = []
    if additional_columns is None:
        additional_columns = {}

    fields = pydantic_model.__fields__
    columns = {
        '__tablename__': table_name,
        'id': Column(Integer, primary_key=True, autoincrement=True)
    }
    
    for field_name, field in fields.items():
        if field_name in exclude_fields:
            continue
        field_type = field.outer_type_
        columns[field_name] = Column(type_mapping[field_type], nullable=False)
    
    # Add additional columns for relationships or other special cases
    columns.update(additional_columns)
    
    # Create the new model class
    model = type(table_name, (base,), columns)
    return model

# Dynamically create SQLAlchemy models without relationships
TEvaluationResponse = create_sqlalchemy_model(
    EvaluationResponse, 
    Base, 
    'evaluation_responses', 
    exclude_fields=['evaluation_by_model']
)
TEvaluationByModel = create_sqlalchemy_model(
    EvaluationByModel, 
    Base, 
    'evaluation_by_model'
)

# Define relationships after both classes have been created
TEvaluationResponse.evaluations = relationship("TEvaluationByModel", back_populates="response")
TEvaluationByModel.response_id = Column(Integer, ForeignKey('evaluation_responses.id'))
TEvaluationByModel.response = relationship("TEvaluationResponse", back_populates="evaluations")

# Create an engine and a session
engine = create_engine('sqlite:///evaluations.db')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)


In [39]:
def save_evaluation_response(session, evaluation_response):

    response_dict = evaluation_response.dict(exclude={"evaluation_by_model"})
    print(response_dict)
    response = TEvaluationResponse(
        **response_dict  # Unpack the dictionary into keyword arguments
    )
    session.add(response)
    session.flush()

    for eval_model in evaluation_response.evaluation_by_model:
        model_evaluation = TEvaluationByModel(
            response_id=response.id,  # Use the generated ID
            **eval_model.dict()  # Unpack the dictionary into keyword arguments
        )
        session.add(model_evaluation)
    
    session.commit()

In [40]:
session = Session()
save_evaluation_response(session, response)
session.close()

{'instruction': 'ㅁㅁㅁ', 'accuracy': '정확성(Accuracy) 평가 기준', 'relevance': '관련성(Relevance) 평가 기준', 'fluency': '유창성(Fluency) 평가 기준', 'completeness': '완전성(Completeness) 평가 기준', 'overall': 'gpt-4o 모델이 가장 정확하고 관련성 있으며 유창하고 완전한 답변을 제공함.'}
